# Building the Dashboard with Dash

We're finally ready to start building our dashboard with Dash, a free and open source Python library created and maintained by the company Plotly. Dash provides many tools to display data visually as well as allow users to interact with it.

Dash is built directly on top of Flask, one of the most popular web frameworks for Python, and react.js, an open source JavaScript library originally created by Facebook. Dash directly integrates with the plotly library. It does not make any plots on its own, but is equipped to present any plotly figure within it.

## Parts of a Dash application

There are two parts of a Dash application:

* Layout
* Interactivity (Callbacks)

### Layout

The layout describes the physical components of the application. There are two broad types of layout components - **HTML elements** and **dash components**. The HTML elements are the exact same elements we covered in the previous notebook. We will not be writing HTML directly, but accessing the elements as Python classes from the **dash_html_components** module. 

The dash components combine together multiple HTML elements, CSS, and JavaScript into a single component. Examples of these include dropdown menus, checklists, tabs, placeholders for plotly figures, and sliders. They are accessed as single Python classes from the **dash_core_components** module. Additionally, there is an entire separate library, **dash_table**, that provides functionality for interactive data tables.

### Interactivity (Callbacks)

The interactivity is the other part of the application, and is the code that makes changes to the layout components when a particular event occurs. Almost any event that occurs within a dash application can be mapped to some function that changes one of the layout components. These functions that are triggered are named **callbacks.** For example, when a country is clicked in our data table, three graphs are updated to show the clicked country's information.

## Beginning a dash application

Typically, dash applications are written in a separate file, such as `dashboard.py` in our case. For purposes of instruction, the dashboard will be written in this Jupyter Notebook. When writing real dashboard applications, I strongly recommend beginning them in a normal text file and using an editor such as [Sublime Text][0] or [Visual Studio Code][1]. 

### Installing JupyterDash

Even though this tutorial takes place in a notebook, the code will be virtually identical as it would be in a separate text file. I recommend opening up `dashboard.py` in a separate editor now so that you can reference it during the tutorial. The only difference is that we'll need to install a new library for it to work properly. Plotly released the JupyterDash library to help those develop Dash within a notebook. This library is only needed for the tutorial and not for our production project, so we won't add it to our virtual environment. Install it now using either pip or conda.

```bash
pip install jupyter-dash==0.4

```

```bash
conda install -c conda-forge -c plotly jupyter-dash=0.4
```

### Minimal dashboard

Your dashboard will need a minimum of three lines of code to run - one to create the application with `JupyterDash`, another to set the `layout` attribute, and lastly, to execute the `run_server` method with `mode` set to `"inline"`, so it displays in the notebook. Here, we assign the layout to be a single HTML element, an h2 header and limit the height of the entire dashboard.

[0]: https://www.sublimetext.com/
[1]: https://code.visualstudio.com/

In [ ]:
from jupyter_dash import JupyterDash
import dash_html_components as html

app = JupyterDash(__name__)
app.layout = html.H2('Coronavirus Forecasting Dashboard')
app.run_server(mode='inline', height=100)

## HTML elements in dash

Nearly all normal HTML elements are available as Python classes after importing `dash_html_components` and aliasing it as `html`. Element names have their first letter capitalized. The first argument for every HTML class in dash is the element's content, formally given the parameter name **children**. The content is often a string for paragraph, header, and anchor elements. Use a list when the content consists of multiple other HTML elements (which is often the case for divs).  

Below, we create an anchor element and then place it and the header inside of a div. Dash does not allow you to assign the `layout` attribute to a list of elements. You must set it to a single HTML element, so we are forced to use a div to contain all of the elements.

In [ ]:
title = html.H2('Coronavirus Forecasting Dashboard')
link = html.A('Visit live dashboard', href='https://coronavirus.dunderdata.com')
layout = html.Div([title, link])

app = JupyterDash(__name__)
app.layout = layout
app.run_server(mode='inline', height=100)

### Adding CSS

In our actual project, most CSS will be kept in the external stylesheet, style.css. For this tutorial, it will be easier to see the CSS together with elements. Unfortunately, the `dash_html_components` library does not have a style element. However, CSS may be added with the `style` parameter, setting it to a dictionary with each property mapped to its value, with the caveat that attribute names are written in **camelCase**. Also, you cannot set the `class` or `id` within the `style` dictionary, but must use the `id` and `className` parameters instead. The following list summarizes the differences writing HTML in dash:

* Nearly all normal HTML elements are available in `dash_html_components`
* Element names are capitalized
* Attributes are the same and available as parameters
* Set the `style` parameter to a dictionary to apply CSS
* Properties in the style dictionary are **camelCased**
* Set id and class with parameters `id` and `className`

Here, we use the same layout from above, but add style to the header.

In [ ]:
title = html.H2('Coronavirus Forecasting Dashboard', 
                style={
                    'backgroundColor': 'tan',
                    'fontFamily': 'verdana',
                    'textAlign': 'center'
                }
               )
link = html.A('Visit live dashboard', href='https://coronavirus.dunderdata.com')
layout = html.Div([title, link])

app = JupyterDash(__name__)
app.layout = layout
app.run_server(mode='inline', height=100)

## Creating a data table

A separate package, `dash_table`, contains the class `DataTable` to create interactive tables of data. Visit the [official documentation][0] for full coverage. There are many parameters that you can set during construction of the table:

* `data` - List of dictionaries where each list represents one row. 
    * `df.to_dict('records')` converts a pandas DataFrame into the proper data structure
* `columns` - List of dictionaries where each item represents information (name, id, type, format, etc...) on one column. 
    * `[{'name': name, 'id': name} for name in df.columns]`
* `sort_action` - Allow sorting of columns by setting to string `'native'`
* `active_cell` - Initial active cell - will be highlighted - `{"row": 0, "column": 0}`
* `style_table` - CSS for `table` HTML element (not for individual rows, columns, or cells)
* `style_cell` - CSS for ALL cells
* `style_header` - CSS for only the header (columns)
* `style_data` - CSS for only the data
* `style_data_conditional` - CSS based on if/else condition
    * `{"if": {"column_id": first_col}, "width": "120px", "textAlign": "left"}`
* `fixed_rows` - Keep the first `n` rows fixed when scrolling down. Mainly used to keep the column names on top.
    * `{ 'headers': True, 'data': n }`

[0]: https://dash.plotly.com/datatable

Let's read in the summary table as a pandas DataFrame placing the group (either "world" or usa") in the index. It has one row per area for the "current" date.

In [ ]:
import pandas as pd
SUMMARY = pd.read_csv("data/summary.csv", index_col="group", parse_dates=["date"])
SUMMARY.head(3)

This is the data we'd like to place within a dash data table. Before doing so, we'll select a subset of columns for just the world group and change the area column to "Country".

In [ ]:
group = "world"
used_columns = ["area", "Deaths", "Cases", "Deaths per Million", "Cases per Million"]
df = SUMMARY.loc[group, used_columns]
df = df.rename(columns={"area": "Country"})
df.head(3)

Now that we have our data, we need to provide column info to dash by creating a list of dictionaries. At a minimum, the `name` (visible label for column) and `id` (internal identification) must be present. We set both the name and id to the column name. The `type` ("any", "numeric", "text", or "datetime") is given along with setting `format` to a dictionary that uses the key `specifier` to set the formatting (based on [D3 format][1]). The first column is the area, which is a text column. We iterate to append all of the other columns, which are numeric.

[1]: https://github.com/d3/d3-format

In [ ]:
columns = [{"name": "Country", 
            "id": "Country", "type": "text"}]
for name in df.columns[1:]:
    col_info = {
        "name": name,
        "id": name,
        "type": "numeric",
        "format": {'specifier': ','}
    }
    columns.append(col_info)

We sort the DataFrame by deaths and then convert it to a list of dictionaries, which is necessary for the dash table.

In [ ]:
data = df.sort_values("Deaths", ascending=False).to_dict("records")
data[:3]

We are now set to create our data table. We use `active_cell` to highlight and make the first cell active. We also use conditional styling to underline values in the first column and turn the cursor into a pointer to inform the user that it is clickable. 


### Note on table height

Unfortunately, behavior for table height is not consistent and requires us to set it using both `minHeight` and `height` CSS properties when using `fixed_rows`/`fixed_columns`. We set it using the units `vh`, which stands for **viewport height**, where 1vh represents 1% of the viewable height of the screen. Below, we set it to 80vh so that the table takes up 80% of the viewable screen height.

In [ ]:
from dash_table import DataTable
world_table = DataTable(
    id=f"{group}-table",
    columns=columns,
    data=data,
    fixed_rows={"headers": True},
    active_cell={"row": 0, "column": 0},
    sort_action="native",
    derived_virtual_data=data,
    style_table={
        "minHeight": "80vh",
        "height": "80vh",
        "overflowY": "scroll"
        },
    style_cell={
        "whitespace": "normal",
        "height":"auto",
        "fontFamily": "verdana"
    },
    style_header={
        "textAlign": "center",
        "fontSize": 14
    },
    style_data={
        "fontSize": 12
    },
    style_data_conditional=[
        {
            "if": {"column_id": "Country"},
            "width": "120px",
            "textAlign": "left",
            "textDecoration": "underline",
            "cursor": "pointer"
        },
        {
            "if": {"row_index": "odd"}, 
            "backgroundColor": "#fafbfb"
        }
    ],
)
layout = html.Div([title, world_table])

app = JupyterDash(__name__)
app.layout = layout
app.run_server(mode='inline', height=500)

The function below encapsulates all of our work from above and is nearly identical to the one found in `dashboard.py`.

In [ ]:
def create_table(group):
    used_columns = [
        "area",
        "Deaths",
        "Cases",
        "Deaths per Million",
        "Cases per Million",
    ]
    df = SUMMARY.loc[group, used_columns]
    first_col = "Country" if group == "world" else "State"
    df = df.rename(columns={"area": first_col})

    columns = [{"name": first_col, "id": first_col}]
    for name in df.columns[1:]:
        col_info = {
            "name": name,
            "id": name,
            "type": "numeric",
            "format": {'specifier': ','},
        }
        columns.append(col_info)

    data = df.sort_values("Deaths", ascending=False).to_dict("records")
    return DataTable(
        id=f"{group}-table",
        columns=columns,
        data=data,
        active_cell={"row": 0, "column": 0},
        fixed_rows={"headers": True},
        sort_action="native",
        derived_virtual_data=data,
        style_table={
            "minHeight": "80vh",
            "height": "80vh",
            "overflowY": "scroll",
            "borderRadius": "0px 0px 10px 10px",
        },
        style_cell={
            "whiteSpace": "normal",
            "height": "auto",
            "fontFamily": "verdana",
        },
        style_header={
            "textAlign": "center",
            "fontSize": 14,
        },
        style_data={
            "fontSize": 12,
        },
        style_data_conditional=[
            {
                "if": {"column_id": first_col},
                "width": "120px",
                "textAlign": "left",
                "textDecoration": "underline",
                "cursor": "pointer",
            },
            {
                "if": {"row_index": "odd"}, 
                "backgroundColor": "#fafbfb"
            }
        ],
    )

Let's use it to create the USA data table and recreate the world table as well. These tables will be accessible from their respective tab.

In [ ]:
world_table = create_table("world")
usa_table = create_table("usa")

## Dash core components

The [dash_core_components library][1], aliased as `dcc`, contains many interactive widgets such as checklists, dropdown menus, buttons, tabs, and others. You can think of these widgets as a collection of HTML, CSS, and JavaScript wrapped into a single Python class that is ready to use.

###  Creating tabs

Let's create two tabs, one for each of the world and USA. To do so we'll need both the `Tabs` and `Tab` classes from `dash_core_components`. The `Tabs` component is the container for each individual `Tab`, which is the container for the content (data table in this example). Here, we create two individual tabs, using the data table as the content. Dash allows you to provide a normal class name, and another one for when it is selected. This enables us to apply different styling based on which tab is selected.

[1]: https://dash.plotly.com/dash-core-components

In [ ]:
import dash_core_components as dcc

def create_tab(content, label, value):
    return dcc.Tab(
        content,
        label=label,
        value=value,
        id=f"{value}-tab",
        className="single-tab",
        selected_className="single-tab--selected",
    )

world_tab = create_tab(world_table, "World", "world")
usa_tab = create_tab(usa_table, "US States", "usa")

We pass the individual tabs as a list to the `Tabs` component and update our layout to show the tabs with the tables.

In [ ]:
table_tabs = dcc.Tabs(
    [world_tab, usa_tab], 
    className="tabs-container", 
    id="table-tabs",
    value="world"
)
layout = html.Div([title, table_tabs])

app = JupyterDash(__name__)
app.layout = layout
app.run_server(mode='inline', height=500)

## Adding plotly figures with `dcc.Graph`

Plotly figures must be placed within the `dcc.Graph` component in order to be added to the dashboard. Before we make the graphs, let's read in all of the data, putting the group, area, and date in the index.

In [ ]:
ALL_DATA = pd.read_csv("data/all_data.csv", 
                       index_col=["group", "area", "date"], 
                       parse_dates=["date"]).sort_index()
ALL_DATA.head()

We made all of the figures for our dashboard in the chapter covering plotly visualizations. The cell below defines the following functions:

* `create_figures` - creates three empty plotly figures with two plots each
* `make_cumulative_graphs` - cumulative line graphs of total deaths/cases
* `make_daily_graphs` - daily bar chart of deaths/cases
* `make_weekly_graphs` - aggregated weekly totals of deaths/cases
* `create_graphs` - runs the above four functions and returns three completed figures.

In [ ]:
from plotly.subplots import make_subplots
from plotly.colors import qualitative
COLORS = qualitative.T10[:2]
LAST_DATE = SUMMARY['date'].iloc[-1]
FIRST_PRED_DATE = LAST_DATE + pd.Timedelta('1D')

def create_figures(title, n=3):
    figs = []
    annot_props = {"x": 0.1, "xref": "paper", "yref": "paper", "xanchor": "left",
                   "showarrow": False, "font": {"size": 18},}
    for _ in range(n):
        fig = make_subplots(rows=2, cols=1, vertical_spacing=0.1)
        fig.update_layout(
            title={"text": title, "x": 0.5, "y": 0.97, "font": {"size": 20}},
            annotations=[
                {"y": 0.95, "text": "<b>Deaths</b>"},
                {"y": 0.3, "text": "<b>Cases</b>"},
            ],
            margin={"t": 40, "l": 50, "r": 10, "b": 0},
            legend={"x": 0.5, "y": -0.05, "xanchor": "center", "orientation": "h",
                    "font": {"size": 15}})
        fig.update_traces(showlegend=False, row=2, col=1)
        fig.update_traces(hovertemplate="%{x} - %{y:,}")
        fig.update_annotations(annot_props)
        figs.append(fig)
    return figs

def make_cumulative_graphs(fig, df_dict, kinds):
    for row, kind in enumerate(kinds, start=1):
        for i, (name, df) in enumerate(df_dict.items()):
            fig.add_scatter(x=df.index, y=df[kind], mode="lines+markers", 
                            showlegend=row==1, line={"color": COLORS[i]}, 
                            name=name, row=row, col=1)

def make_daily_graphs(fig, df_dict, kinds):
    for row, kind in enumerate(kinds, start=1):
        for i, (name, df) in enumerate(df_dict.items()):
            fig.add_bar(x=df.index, y=df[kind], marker={"color": COLORS[i]},
                        showlegend=row==1, name=name, row=row, col=1)

def make_weekly_graphs(fig, df_dict, kinds):
    offset = "W-" + LAST_DATE.strftime("%a").upper()
    df_dict = {name: df.resample(offset, kind="timestamp", closed="right")[kinds].sum()
               for name, df in df_dict.items()}

    for row, kind in enumerate(kinds, start=1):
        for i, (name, df) in enumerate(df_dict.items()):
            fig.add_scatter(x=df.index, y=df[kind], mode="lines+markers",
                            showlegend=row==1, line={"color": COLORS[i]},
                            name=name, row=row, col=1)
            
def create_graphs(group, area):
    df = ALL_DATA.loc[(group, area)]
    df_dict = {"actual": df.loc[:LAST_DATE], "prediction": df.loc[FIRST_PRED_DATE:]}
    kinds = ["Deaths", "Cases"]
    new_kinds = ["Daily Deaths", "Daily Cases"]
    figs = create_figures(area)
    make_cumulative_graphs(figs[0], df_dict, kinds)
    make_daily_graphs(figs[1], df_dict, new_kinds)
    make_weekly_graphs(figs[2], df_dict, new_kinds)
    return figs

Let's choose one area to make the three graphs.

In [ ]:
figs = create_graphs('usa', 'Texas')

Here are the cumulative line graphs for cases in deaths as a plotly figure. This is independent of dash at this point.

In [ ]:
figs[0]

Daily bar graph:

In [ ]:
figs[1]

Weekly aggregate line graph:

In [ ]:
figs[2]

We set the `figure` parameter of `dcc.Graph` to be one of the figures and then place that object in our layout. This new object would appear under the table by default, since it's a block-level element. To display it to the side of the table, we place the tabs and graph within a div and set its `display` property to `grid`. A 2 x 2 grid is created using the following `gridTemplateAreas`.

```css
"tables graphs"
"tables maps"
```

Each column is set to be 50% of the viewport width, `vw`. The graph is given the `gridArea` of "graphs", while the style of `table_tables` is updated so that it's `gridArea` is "tables". Lastly, `columnGap` is used to separate the left and right columns by 10 pixels.

In [ ]:
cumulative_graph = dcc.Graph(figure=figs[0], style={'gridArea': "graphs"})
table_tabs.style = {'gridArea': "tables"}
container = html.Div([table_tabs, cumulative_graph], 
                     style={'display': 'grid',
                            'gridTemplateAreas': '"tables graphs" "tables maps"',
                            'gridTemplateColumns': "50vw 50vw",
                            'columnGap': '10px'})
layout = html.Div([title, container])

app = JupyterDash(__name__)
app.layout = layout
app.run_server(mode='inline', height=600)

In the actual dashboard, we have tabs to cycle through each graph. We'll duplicate the procedure from above, adding each graph to its own tab.

In [ ]:
cumulative_graph = dcc.Graph(figure=figs[0], id="cumulative-graph")
daily_graph = dcc.Graph(figure=figs[1], id="daily-graph")
weekly_graph = dcc.Graph(figure=figs[2], id="weekly-graph")

cumulative_tab = create_tab(cumulative_graph, "Cumulative", "cumulative")
daily_tab = create_tab(daily_graph, "Daily", "daily")
weekly_tab = create_tab(weekly_graph, "Weekly", "weekly")

graph_tabs = dcc.Tabs(
    [cumulative_tab, daily_tab, weekly_tab],
    className="tabs-container",
    id="graph-tabs",
    value="cumulative",
    style={'gridArea': 'graphs', 'margin': '0px'}
)

container = html.Div([table_tabs, graph_tabs], 
                     style={'display': 'grid',
                            'gridTemplateAreas': '"tables graphs" "tables maps"',
                            'gridTemplateColumns': "50vw 50vw",
                            'columnGap': '10px'})
layout = html.Div([title, container])

app = JupyterDash(__name__)
app.layout = layout
app.run_server(mode='inline', height=600)

## Adding the maps

The last main component of the dashboard are the world and USA maps. Here, we copy the code from a previous chapter that creates the maps with custom hover text.

In [ ]:
import plotly.graph_objects as go

def hover_text(x):
    name = x["area"]
    deaths = x["Deaths"]
    cases = x["Cases"]
    deathsm = x["Deaths per Million"]
    casesm = x["Cases per Million"]
    return (
        f"<b>{name}</b><br>"
        f"Deaths - {deaths:,.0f}<br>"
        f"Cases - {cases:,.0f}<br>"
        f"Deaths per Million - {deathsm:,.0f}<br>"
        f"Cases per Million - {casesm:,.0f}<br>"
    )

def create_map(group, radio_value):
    df = SUMMARY.loc[group].query("population > 0.5")
    lm = None if group == "world" else "USA-states"
    proj = "robinson" if group == "world" else "albers usa"

    fig = go.Figure()
    fig.add_choropleth(
        locations=df["code"],
        z=df[radio_value],
        zmin=0,
        locationmode=lm,
        colorscale="orrd",
        marker_line_width=0.5,
        text=df.apply(hover_text, axis=1),
        hoverinfo="text",
        colorbar=dict(len=0.6, x=1, y=0.5),
    )
    fig.update_layout(
        geo={
            "lataxis": {"range": [-50, 68]},
            "lonaxis": {"range": [-130, 150]},
            "projection": {"type": proj},
            "showframe": False,
        },
        margin={"t": 0, "l": 10, "r": 10, "b": 0},
    )
    return fig

Now we create a single map and color it by deaths per million.

In [ ]:
fig_map = create_map('world', 'Deaths per Million')
fig_map

## Adding radio buttons above the map

Our dashboard allows the user to choose the coloring of the map with four radio buttons. The dash core components library does have radio buttons, but does not supply an easy way to change the style whenever the radio button is checked.

### Dash Bootstrap Components

The third-party library [Dash Bootstrap Components][1] has many components for dash that are built using [Bootstrap][2], a very popular CSS and JavaScript library containing nicely styled HTML components. Here, we create the radio buttons, which require a list of dictionaries setting the visible `label` and internal `value` to strings for all options. The `labelCheckedStyle` allows us to apply a separate style to the checked label. We create a dashboard with just the radio buttons to view them before adding them above the map.

[1]: https://dash-bootstrap-components.opensource.faculty.ai/
[2]: https://getbootstrap.com/

In [ ]:
import dash_bootstrap_components as dbc

radio_items = dbc.RadioItems(
    options=[
        {"label": "Deaths", "value": "Deaths"},
        {"label": "Cases", "value": "Cases"},
        {"label": "Deaths per Million", "value": "Deaths per Million"},
        {"label": "Cases per Million", "value": "Cases per Million"},
    ],
    value="Deaths",
    id="map-radio-items",
    style={'display': 'flex', 
           'justifyContent': 'space-evenly', 
           'backgroundColor': '#212529', 
           'color': '#798d8f'},
    labelCheckedStyle={'fontWeight': 800, 'color': 'white'}
)


app = JupyterDash(__name__)
app.layout = radio_items
app.run_server(mode='inline', height=50)

The radio buttons and map are wrapped in a div and added to the bottom right corner. The buttons will have no effect until we add the interactivity. Note that we had to set the style of each graph component in order to set the height. This is a bug in dash as the graph does not expand or contract to fit the size of its container.

In [ ]:
cumulative_graph.style = {'height': '40vh'}
daily_graph.style = {'height': '40vh'}
weekly_graph.style = {'height': '40vh'}

map_graph = dcc.Graph(figure=fig_map, id="map-graph", style={'height': '40vh'})
map_div = html.Div([radio_items, map_graph], style={'gridArea': 'maps'})
container = html.Div([table_tabs, graph_tabs, map_div], 
                     style={'display': 'grid',
                            'gridTemplateAreas': '"tables graphs" "tables maps"',
                            'gridTemplateColumns': "50vw 50vw",
                            'gridTemplateRows': "50vh 40vh",
                            'columnGap': '10px',
                            })
layout = html.Div([title, container])

app = JupyterDash(__name__)
app.layout = layout
app.run_server(mode='inline', height=800)

## Interactivity using callbacks

The **layout** of our dashboard is now complete. We are ready to move on to the second half of our application, the **interactivity**. Code for the interactivity MUST appear after `app.layout` has been set or you'll get an error. The interactivity is user-defined functions named callbacks that get triggered by some event on the dashboard, usually a click.

### Writing a callback

A summary of the callback function is provided below:

* `app.callback` method decorates the function and takes three main arguments:
    * Outputs - A list of layout components to be changed
    * Inputs - A list of layout components that trigger the function
    * States - A list of non-triggered layout components to pass to the function
* Function defined with one parameter for every input
* Function must return the number of outputs

### `Output`, `Input`, and `State`

You will use the objects `Output`, `Input`, and `State` to describe the arguments to your callback function, and the return values. All three objects are found in the `dash.dependencies` module and all require two arguments, the **id** of the component, and the targeted **property**. Here are some examples:

* `Output("cumulative-graph", "figure")` - informs dash that the component with id of "cumulative-graph" should have its "figure" property replaced with a returned value.
* `Input('world-table', 'active_cell')` - informs dash to trigger the function whenever the `active_cell` property of the component with id "world-table" is changed.
* `State('world-table', 'derived_virtual_data')` - tells dash to pass the value of the property `derived_virtual_data` of the component with id "world-table" to the function. `State` does not trigger the function. It's just extra data to pass to the function whenever it does get triggered.

Take a look at the interactivity below. We have a single function that is decorated by `app.callback`. The function name can be anything you desire. This function changes three components (the "figure" property of each graph in the upper right corner). From the inputs section, we see that it gets triggered when the active cell changes in the world table. Whenever this function gets triggered, it also gets passed the value of `derived_virtual_data` from the world table.

### `data` vs `derived_virtual_data`

You might be wondering what the difference is between the `data` and `derived_virtual_data` properties from a dash data table. They are both lists containing a dictionary for every row of data. But, the `data` property is **static** and never changes. If the table gets sorted by the user, the list from `data` remains the same. The `derived_virtual_data` is **dynamic** and always matches the data that the user sees. This is why we use it and not the value for the `data` property.

### All callbacks called when dashboard starts

By default, all callbacks are called at the start of the application. This may seem like undesired behavior, but it can actually save you from repeating code, as you don't have to set the components that will be changed during the callback. 

Our function below will be called by dash whenever the active cell changes, and will be passed two arguments. The parameter names for the function are completely arbitrary, and can be anything you choose. The order of the arguments passed comes directly from the order of the inputs and states in the decorator.

Our function receives the default values for the `active_cell` and `derived_virtual_data`, in that order. The text value of `active_cell` is not provided, just a dictionary of row and column integers. We use this info to find the actual value from the list of dictionaries from `derived_virtual_data`. Once we get the country name, we call the `create_graphs` function, which returns three figures, replacing the "figure" property of each of the `dcc.Graph` objects. To help understand what is passed to the function, a print statement is added to show some variable values.

In [ ]:
from dash.dependencies import Output, Input, State

app = JupyterDash(__name__)
app.layout = layout

@app.callback(
    [
        Output("cumulative-graph", "figure"),
        Output("daily-graph", "figure"),
        Output("weekly-graph", "figure"),
    ],
    [
        Input('world-table', 'active_cell')
    ],
    [
        State('world-table', 'derived_virtual_data')
    ]
)
def change_area_graphs(world_cell, world_data):
    """
    Change the all three graphs in the upper right hand corner of the app
    
    Parameters
    ----------
    world_cell : dict with keys `row` and `cell` mapped to integers of cell location
    
    world_data : list of dicts of one country per row.
                     Has keys Country, Deaths, Cases, Deaths per Million, Cases per Million
                     
    Returns
    -------
    List of three plotly figures, one for each of the `Output`
    """
    row_number = world_cell["row"]
    row_data = world_data[row_number]
    country = row_data['Country']
    print("active_cell", world_cell, 
          "\nrow_number", row_number, 
          "\nrow_data", row_data, 
          "\ncountry", country)
    return create_graphs('world', country)
    
app.run_server(mode='inline', height=800)

### Single Callback for updating both World and USA data

A component may only appear as an output in exactly **one** callback. This means we cannot write separate callbacks for the world and USA tables, as they update the same three graphs in the upper right corner. To work around this limitation, we redefine our callback to have a total of three input triggers - both data tables and the tab above them. If the active cell from either of the tables change or the tab switches, the function will be triggered. It gets passed those three inputs plus the underlying list of dictionaries (the `derived_virtual_data`) from both tables.

The body of the function is updated to perform some logic to determine whether the world or USA tab is active. Since any cell can be the active cell, it only updates the graph when the first column (index equal to 0) is active. It's also possible that there is no active cell, making its value `None`. This happens whenever a sort is performed. The statement `if cell and cell["column"] == 0` ensures that there is an active cell and it's in the first column. If this check fails, we must raise the `PreventUpdate` exception (that you must import from the `dash.exceptions` module) as dash expects three figures to be returned. This is a useful exception to know about, as there will be times a user triggers a callback, but the desired behavior is to not change anything.

In [ ]:
from dash.exceptions import PreventUpdate

app = JupyterDash(__name__)
app.layout = layout

@app.callback(
    [
        Output("cumulative-graph", "figure"),
        Output("daily-graph", "figure"),
        Output("weekly-graph", "figure"),
    ],
    [
        Input("world-table", "active_cell"),
        Input("usa-table", "active_cell"),
        Input("table-tabs", "value"),
    ],
    [
        State("world-table", "derived_virtual_data"),
        State("usa-table", "derived_virtual_data"),
    ],
)
def change_area_graphs(world_cell, usa_cell, group, world_data, usa_data):
    area, cell, data = "Country", world_cell, world_data
    if group == "usa":
        area, cell, data = "State", usa_cell, usa_data
    if cell and cell["column"] == 0:
        country_state = data[cell["row"]][area]
        return create_graphs(group, country_state)
    else:
        raise PreventUpdate
        
app.run_server(mode='inline', height=800)

## Callback to change the map

We need to add one more callback to change the map and/or its coloring.  This function gets triggered when either the tabs above the table change or one of the radio items is clicked. We return a single figure using our `create_map` function defined above.

In [ ]:
app = JupyterDash(__name__)
app.layout = layout

@app.callback(
    [
        Output("cumulative-graph", "figure"),
        Output("daily-graph", "figure"),
        Output("weekly-graph", "figure"),
    ],
    [
        Input("world-table", "active_cell"),
        Input("usa-table", "active_cell"),
        Input("table-tabs", "value"),
    ],
    [
        State("world-table", "derived_virtual_data"),
        State("usa-table", "derived_virtual_data"),
    ],
)
def change_area_graphs(world_cell, usa_cell, group, world_data, usa_data):
    area, cell, data = "Country", world_cell, world_data
    if group == "usa":
        area, cell, data = "State", usa_cell, usa_data
    if cell and cell["column"] == 0:
        country_state = data[cell["row"]][area]
        return create_graphs(group, country_state)
    else:
        raise PreventUpdate

@app.callback(
    Output("map-graph", "figure"),
    [
        Input("table-tabs", "value"), 
        Input("map-radio-items", "value")
    ],
)
def change_map(group, radio_value):
    return create_map(group, radio_value)

app.run_server(mode='inline', height=800)

### Differences between this tutorial and `dashboard.py`

Open the `dashboard.py` file and review the code. It is largely the same as it was presented in this tutorial, however, there are a few differences:

* The dash application is instantiated with `Dash` from the library `dash` and not `JupyterDash`
* All of the CSS (except for the data tables) has moved to assets/style.css
* The CSS for Bootstrap (and any other external CSS) must be linked using the `external_stylesheets` parameter when instantiating `Dash`. It is set to a list of URLs containing CSS.
* A new column in our CSS grid is added on the left. Four bootstrap "cards" from `dash_bootstrap_components` are placed in this column.
* A navigation bar is added to the top of the page using `dash_bootstrap_components`
* CSS is added to target screens with a width less than 1000px using a media query.
    * Take a look at the bottom of assets/style.css. You'll see the following selector 
        * `@media only screen and (max-width: 1000px)`

## Dash Summary

* Dash vs Plotly 
    * Dash builds the dashboard application with HTML components and provides interactivity with callbacks
    * Dash does not do visualization
    * Plotly creates all visualizations
    * Plotly visualizations must be placed in a `dcc.Graph` component
* Two parts of a dash application
    * **Layout**
        * Dash HTML Components - nearly all regular HTML components
        * Dash Core Components - interactive widgets
        * Dash Data Tables - two dimensional tables
        * Dash Bootstrap Components - third-party library with more widgets styled with Bootstrap
        * Must set `app.layout` to a dash component to finalize layout
    * **Interactivity**
        * Functions triggered by user events
        * Functions receive component values as inputs
        * Functions return new component values as outputs
        * Interactivity must come after `app.layout` is set
* Dash HTML Components
    * Separate library - `dash_html_components` aliased as `html`
    * Uses same tag names as regular HTML, but capitalized
    * First parameter is content
    * Provide id and class as `id` and `className` parameters
    * Can provide CSS with `style` parameter, but better to do so in `assets/style.css`
* Dash Core Components
    * Separate library - `dash_core_components` aliased as `dcc`
    * Many widgets that bundle together HTML/CSS and JavaScript
    * Dropdown menus, radio buttons, tabs, and more
    * Use `dcc.Graph` to add plotly figure
* Dash Data Tables
    * Separate Library - `dash_table`
    * Provide data as a list of dictionaries. Use `df.to_dict('records')`
    * Set style for whole table, each cell, just the headers, or just the data with `style_*` parameters
* Dash Bootstrap Components
    * Third-party library - `dash_bootstrap_components` - aliased as dbc
    * Must set `external_stylesheets` in `Dash` constructor to a list of URLs of the specific flavor of Bootstrap you want.
    * Many extra components not provided by `dash_core_components`
* Callback functions
    * Must be defined after `app.layout` is set
    * Function and parameter names are arbitrary
    * Decorate functions with `app.callback`
    * Pass `app.callback` a list of the outputs, inputs, and states
    * Use `Input`, `Output`, and `State` from `dash.dependencies` to target the component and its property
        * Example: `Input("component-id", "component-property")`
    * Function must return a value for each of the outputs
    * Raise the `dash.exceptions.PreventUpdate` exception if you don't want the function to update any component
    * Every callback is called when the app first launches
    * Each component may appear as an output in exactly one callback
        * May need to combine many triggers into one function to work around this limitation
    * Each component may appear as an input in ANY number of callbacks